In [1]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    pipeline,
)
import torch
from datasets import (
    load_dataset,
    Dataset,
    DatasetDict,
)

import random
from tqdm import tqdm
import gc
import pandas as pd

# set gpu
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

device= "cuda" if torch.cuda.is_available() else "cpu"
batch_size = 4

/home/infres/abounhar/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ############### to create the bench ################

def prepare_causalLM_samples(samples, text_column='ground_truth'):
    """Prepare samples by truncating the text."""
    test_samples = []
    for text in samples[text_column]:
        words = text.split()
        if len(words) < 45:
            # we consider half of the sentence
            text = ' '.join(words[:len(words)//2])
        else:
            # we consider 2/3 of the sentence as the sentence is long
            text = ' '.join(words[:len(words)//3*2])
        test_samples.append(text)
    
    return {"causal_sentence": test_samples}

tarjama_ds = load_dataset("atlasia/TerjamaBench",split="test")

selected_topic = ["religion","idioms","long_sentences","common_phrases","religion","humor"]
length = [15,10,25,10,10]

# select sentences from the dataset
sentences=[]
topics=[]
for topic,l in zip(selected_topic,length):
  # consider only sentences with more than 20 wordataset
  filter_ds = tarjama_ds.filter(
    lambda x: 
      x["topic"]==topic and len(x["Darija"])>20
  )
  sentences += filter_ds["Darija"][:l]
  topics += filter_ds["topic"][:l]
  
# create mask for each sentence
masked_sentences=[]
for sentence in sentences:
  tokens=sentence.split(" ")
  idx=random.randint(0,len(tokens)-1)
  tokens[idx]="<mask>"
  masked_sentences.append(" ".join(tokens))
  
# create a dataset object
dataset = Dataset.from_pandas(
  pd.DataFrame({
      "ground_truth":sentences,
      "masked_sentence":masked_sentences,
      "topic":topics
  })
)

dataset = dataset.map(prepare_causalLM_samples, batched=True, fn_kwargs={"text_column": "ground_truth"})
dataset = dataset.select_columns(['ground_truth', 'masked_sentence', 'causal_sentence', 'topic'])
dataset_dict = DatasetDict({"test": dataset})
# put the dataset to the hub
dataset_dict.push_to_hub('atlasia/LM-Moroccan-Darija-Bench', private=True)

Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.03s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/atlasia/LM-Moroccan-Darija-Bench/commit/b230f418e7b4936e8afd0f78e4456a438c05eefa', commit_message='Upload dataset', commit_description='', oid='b230f418e7b4936e8afd0f78e4456a438c05eefa', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/atlasia/LM-Moroccan-Darija-Bench', endpoint='https://huggingface.co', repo_type='dataset', repo_id='atlasia/LM-Moroccan-Darija-Bench'), pr_revision=None, pr_num=None)

# Fill Mask On TerjamaBench

In [ ]:
dataset = load_dataset("atlasia/LM-Moroccan-Darija-Bench",split="test")

Generating test split: 100%|██████████| 70/70 [00:00<00:00, 13276.11 examples/s]


In [4]:
dataset

Dataset({
    features: ['ground_truth', 'masked_sentence', 'causal_sentence', 'topic'],
    num_rows: 70
})

## Models to test

In [6]:
# load models function
def load_model(model_name,device):
    pipe = pipeline(
        task="fill-mask",
        model=model_name,
        device=device,
    )
    return pipe

In [5]:
MODEL_LIST=[
  "BounharAbdelaziz/XLM-RoBERTa-Morocco",
  "SI2M-Lab/DarijaBERT",
  "BounharAbdelaziz/ModernBERT-Morocco",
  "google-bert/bert-base-multilingual-cased",
  "FacebookAI/xlm-roberta-large",
  "aubmindlab/bert-base-arabertv02",
]

In [7]:
target_name="<mask>"
for model_name in tqdm(MODEL_LIST):
  # load model
  model = load_model(model_name, device)
  # replace the mask token with the model specific mask token
  name = model.tokenizer.all_special_tokens[-1]
  dataset = dataset.map(
    lambda x:
      {"masked_sentence":x.replace(target_name,name)},
      input_columns="masked_sentence"
  )
  target_name=name
  # apply the model on the dataset to get the predictions
  dataset = dataset.map(
    lambda x: {model_name:model(x)},
    input_columns="masked_sentence",
    batched=True,
    batch_size=batch_size,
  )
  # extract the predicted token
  dataset = dataset.map(lambda x: 
    {model_name:x[0]["sequence"]},
    input_columns=model_name,
  )
  gc.collect()
  torch.cuda.empty_cache()
  del model

Map: 100%|██████████| 70/70 [00:00<00:00, 6437.78 examples/s]
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
 17%|█▋        | 1/6 [00:25<02:06, 25.38s/it]The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
Device set to use cuda
 33%|███▎      | 2/6 [00:31<00:55, 13.87s/it]You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initiali

In [8]:
dataset

Dataset({
    features: ['ground_truth', 'masked_sentence', 'causal_sentence', 'topic', 'BounharAbdelaziz/XLM-RoBERTa-Morocco', 'SI2M-Lab/DarijaBERT', 'BounharAbdelaziz/ModernBERT-Morocco', 'google-bert/bert-base-multilingual-cased', 'FacebookAI/xlm-roberta-large', 'aubmindlab/bert-base-arabertv02'],
    num_rows: 70
})

In [9]:
dataset.push_to_hub("atlasia/LM-Moroccan-Darija-Bench", private=True, commit_message="Added predictions of masked LM models")

Uploading the dataset shards: 100%|██████████| 1/1 [00:00<00:00,  1.18it/s]


CommitInfo(commit_url='https://huggingface.co/datasets/atlasia/LM-Moroccan-Darija-Bench/commit/a5a564defb03925861c755f1588dff837f1d9d51', commit_message='Added predictions of masked LM models', commit_description='', oid='a5a564defb03925861c755f1588dff837f1d9d51', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/atlasia/LM-Moroccan-Darija-Bench', endpoint='https://huggingface.co', repo_type='dataset', repo_id='atlasia/LM-Moroccan-Darija-Bench'), pr_revision=None, pr_num=None)

In [10]:
df = dataset.to_pandas()
df.head()

,ground_truth,masked_sentence,causal_sentence,topic,BounharAbdelaziz/XLM-RoBERTa-Morocco,SI2M-Lab/DarijaBERT,BounharAbdelaziz/ModernBERT-Morocco,google-bert/bert-base-multilingual-cased,FacebookAI/xlm-roberta-large,aubmindlab/bert-base-arabertv02
0,لا حول و لا قوة الا بالله,لا حول [MASK] لا قوة الا بالله,لا حول و,religion,لا حول و لا قوة الا بالله,لا حول لشانطي لا قوة الا بالله,لا حول ولا لا قوة الا بالله,لا حولا لا قوة الا بالله,لا حول و لا قوة الا بالله,لا حول و لا قوة الا بالله
1,الله يسمح لينا كاملين,الله يسمح [MASK] كاملين,الله يسمح,religion,الله يسمح لنا كاملين,الله يسمح ليكوم كاملين,الله يسمح لينا كاملين,الله يسمحين كاملين,الله يسمح لنا كاملين,الله يسمح لهم كاملين
2,غير صبر، ربي غايفاجيها,غير صبر، ربي [MASK],غير صبر،,religion,غير صبر، ربي.,غير صبر ، ربي سبحانو,غير صبر، ربي.,غير صبر ، ربي.,غير صبر، ربي,غير صبر ، ربي.
3,الله يسمحلينا من الوالدين,[MASK] يسمحلينا من الوالدين,الله يسمحلينا,religion,اله يسمحلينا من الوالدين,##اله يسمحلينا من الوالدين,منين يسمحلينا من الوالدين,لم يسمحلينا من الوالدين,ما يسمحلينا من الوالدين,لا يسمحلينا من الوالدين
4,لا اله الا الله محمد رسول الله,[MASK] اله الا الله محمد رسول الله,لا اله الا,religion,لا اله الا الله محمد رسول الله,ايدين اله الا الله محمد رسول الله,تبارك اله الا الله محمد رسول الله,الله اله الا الله محمد رسول الله,لا اله الا الله محمد رسول الله,لا اله الا الله محمد رسول الله


In [ ]:
# from google.colab import sheets
# sheet = sheets.InteractiveSheet(df=df)

# Causal-LM On TerjamaBench

In [2]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    pipeline,
)
import torch
from datasets import (
    load_dataset,
    Dataset,
)
import random
from tqdm import tqdm
import gc
import pandas as pd

# set gpu
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

device = "cuda" if torch.cuda.is_available() else "cpu"
batch_size = 16

## Models to test

In [3]:
# load models function
def load_causalLM_model(model_name, device):
    """ Load a causal language model """
    tokenizer = AutoTokenizer.from_pretrained(
        model_name,
        padding_side='left',
    )
    model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
    return model, tokenizer

def generate_text_batch(prompts, model, tokenizer, is_instruct, max_new_tokens=16, temperature=0.7, top_p=0.9, top_k=150, num_beams=4, repetition_penalty=1.5):
    """Generate text in batches using a causal language model."""
    # if is_instruct:
    #     prompts = [f"كمل هاد الجملة : {prompt}" for prompt in prompts]
    inputs = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True).to(model.device)
    outputs = model.generate(
        **inputs, 
        max_new_tokens=max_new_tokens, 
        temperature=temperature, 
        top_p=top_p, 
        do_sample=True,
        repetition_penalty=repetition_penalty,
        num_beams=num_beams,
        top_k=top_k,
        early_stopping=True,
        pad_token_id=tokenizer.pad_token_id, # Explicit pad token
        eos_token_id=tokenizer.eos_token_id, # Explicit eos token
    )
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)

# Generate text in batches
def apply_generation(samples, MODEL_NAME, model, tokenizer, is_instruct=False):
    return {MODEL_NAME: generate_text_batch(samples["causal_sentence"], model, tokenizer, is_instruct)}

In [4]:
MODEL_LIST=[
  "BounharAbdelaziz/Al-Atlas-LLM-0.5B",
  "Qwen/Qwen2.5-0.5B",
  "tiiuae/Falcon3-1B-Base",
  "MBZUAI-Paris/Atlas-Chat-2B", # instruct model
]

In [6]:
# load the dataset
dataset = load_dataset('atlasia/LM-Moroccan-Darija-Bench', split='test')
# loop over the models
for model_name in MODEL_LIST:
    # load model and tokenizer
    model, tokenizer = load_causalLM_model(model_name, device)
    # check if the model is an instruct model
    is_instruct = True if model_name == "MBZUAI-Paris/Atlas-Chat-2B" else False
    # apply the generation
    dataset = dataset.map(
        apply_generation, 
        fn_kwargs={"MODEL_NAME": model_name, "model": model, "tokenizer": tokenizer, "is_instruct": is_instruct}, 
        batched=True,
    )
    # remove the model from memory
    gc.collect()
    torch.cuda.empty_cache()
    del model

Map: 100%|██████████| 70/70 [00:03<00:00, 19.53 examples/s]
Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Map: 100%|██████████| 70/70 [00:09<00:00,  7.41 examples/s]


In [7]:
df = dataset.to_pandas()
df.head()

,ground_truth,masked_sentence,causal_sentence,topic,BounharAbdelaziz/XLM-RoBERTa-Morocco,SI2M-Lab/DarijaBERT,BounharAbdelaziz/ModernBERT-Morocco,google-bert/bert-base-multilingual-cased,FacebookAI/xlm-roberta-large,aubmindlab/bert-base-arabertv02,BounharAbdelaziz/Al-Atlas-LLM-0.5B,Qwen/Qwen2.5-0.5B,tiiuae/Falcon3-1B-Base,MBZUAI-Paris/Atlas-Chat-2B
0,لا حول و لا قوة الا بالله,لا حول [MASK] لا قوة الا بالله,لا حول و,religion,لا حول و لا قوة الا بالله,لا حول لشانطي لا قوة الا بالله,لا حول ولا لا قوة الا بالله,لا حولا لا قوة الا بالله,لا حول و لا قوة الا بالله,لا حول و لا قوة الا بالله,لا حول و لا قوة الا باله العلي العظيم هادشي لي...,لا حول وراءه | جريدة الصباح الجديد\nالرئيسية / أخ,لا حول والله عليه وسلم.\n\n,لا حول و لا قوة إلا بالله\n\nالله يهدي الجميع ...
1,الله يسمح لينا كاملين,الله يسمح [MASK] كاملين,الله يسمح,religion,الله يسمح لنا كاملين,الله يسمح ليكوم كاملين,الله يسمح لينا كاملين,الله يسمحين كاملين,الله يسمح لنا كاملين,الله يسمح لهم كاملين,الله يسمح لينا من الوالدين ديالنا. الله يرحمهم...,الله يسمح لنا ببعض التصرفات التي تجعلنا نستفيد...,الله يسمح الله عليه وسمح الله,الله يسمح لينا. الله يرحمنا و يرحم جميع المسلم...
2,غير صبر، ربي غايفاجيها,غير صبر، ربي [MASK],غير صبر،,religion,غير صبر، ربي.,غير صبر ، ربي سبحانو,غير صبر، ربي.,غير صبر ، ربي.,غير صبر، ربي,غير صبر ، ربي.,غير صبر، غادي نشوفو شنو غادي يوقع من بعد. د,غير صبر، امرأة مصابة بفيروس كورونا تقدم,غير صبر، 2015.\n\n \t 3.\t “,غير صبر، راه غادي يجي نهار لي غادي تكون فيه 100
3,الله يسمحلينا من الوالدين,[MASK] يسمحلينا من الوالدين,الله يسمحلينا,religion,اله يسمحلينا من الوالدين,##اله يسمحلينا من الوالدين,منين يسمحلينا من الوالدين,لم يسمحلينا من الوالدين,ما يسمحلينا من الوالدين,لا يسمحلينا من الوالدين,الله يسمحلينا من هاد المصايب اللي واقعة. الله ...,الله يسمحلينا ويدعوونا إلى الله - صحيفة الاتحا...,الله يسمحلينا عليه وسمحلينا من ال,الله يسمحلينا. الله يرحمنا و يرحم جميع المسلمي...
4,لا اله الا الله محمد رسول الله,[MASK] اله الا الله محمد رسول الله,لا اله الا,religion,لا اله الا الله محمد رسول الله,ايدين اله الا الله محمد رسول الله,تبارك اله الا الله محمد رسول الله,الله اله الا الله محمد رسول الله,لا اله الا الله محمد رسول الله,لا اله الا الله محمد رسول الله,لا اله الا الله انا له وانا اليه راجعون اله يص...,لا اله الايجابي - بوابة الأهرام بوابة الأهرام\nال,لا اله الافعاد على الأمر والتن,لا اله الا الله.\n\nآسف، ولكن ما فهمتش شنو بغي...


In [8]:
dataset.push_to_hub("atlasia/LM-Moroccan-Darija-Bench", private=True, commit_message="Added predictions of causal LM models")

Uploading the dataset shards: 100%|██████████| 1/1 [00:00<00:00,  1.30it/s]


CommitInfo(commit_url='https://huggingface.co/datasets/atlasia/LM-Moroccan-Darija-Bench/commit/04fd79b2f612e4d4d146a1fb30f0add8e20b9f11', commit_message='Added predictions of causal LM models', commit_description='', oid='04fd79b2f612e4d4d146a1fb30f0add8e20b9f11', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/atlasia/LM-Moroccan-Darija-Bench', endpoint='https://huggingface.co', repo_type='dataset', repo_id='atlasia/LM-Moroccan-Darija-Bench'), pr_revision=None, pr_num=None)